# Evaluation of hourly AUC (Area Under Curve) results

From ChatGPT:

| Glycemic State     | Avg Glucose (mg/dL) | AUC (mg·min/dL per hour) |
| ------------------ | ------------------- | ------------------------ |
| Normal             | 80–100              | 4800–6000                |
| Slightly elevated  | 100–140             | 6000–8400                |
| Postprandial spike | 140–180             | 8400–10,800              |
| Hyperglycemia      | >180                | >10,800                  |


In [1]:
%reload_ext autoreload
%autoreload 2

test_data = "../tests/data/example_data_5_subject.csv"

# ChatGPT calculations  

**Average Hourly AUC Per Subject (for test_data)**

| Subject       | Average Hourly AUC (mg*min/dL) | Interpretation                                                                                             |
| ------------- | ------------------ | ---------------------------------------------------------------------------------------------------------- |
| **Subject 1** | 6,241              | ✅ Near-normal range — mild elevation. Likely good glucose control.                                         |
| **Subject 2** | 11,825             | 🔴 High AUC — consistent with **hyperglycemia**, possibly uncontrolled diabetes or postprandial spikes.    |
| **Subject 3** | 8,043              | 🟠 Moderately elevated — suggests elevated average glucose, likely post-meal spikes or insulin resistance. |
| **Subject 4** | 7,041              | 🟡 Slightly elevated — above normal but not extreme. Possibly impaired glucose tolerance.                  |
| **Subject 5** | 9,302              | 🔴 High — consistent with sustained hyperglycemia; could indicate diabetes or poor control.                |


**NOTE:** ChatGpt reports AUC in mg.min/dL, while IGLU operates with mg.h/dL



In [2]:
import pandas as pd

ChatGPT_results = pd.DataFrame(
    {
        'id': ['Subject 1', 'Subject 2', 'Subject 3', 'Subject 4', 'Subject 5'],
        'ChatGPT AUC (mg*min/dL)': [6241, 11825, 8043, 7041, 9302],
    }
)
ChatGPT_results['ChatGPT AUC (mg*h/dL)'] = (ChatGPT_results['ChatGPT AUC (mg*min/dL)'] / 60).round(0)

display(ChatGPT_results)

,id,ChatGPT AUC (mg*min/dL),ChatGPT AUC (mg*h/dL)
0,Subject 1,6241,104.0
1,Subject 2,11825,197.0
2,Subject 3,8043,134.0
3,Subject 4,7041,117.0
4,Subject 5,9302,155.0


# IGLU/IGLU-PY results

**NOTE:** IGLU reports AUC in mg.h/dL


In [ ]:
import sys
from importlib.metadata import version

import iglu_py
import pandas as pd
import rpy2.robjects as ro


In [4]:
# Print versions for future references
print(f"Python version: {sys.version}")
print(f"R version: {ro.r('R.version.string')}")
iglu_version = str(ro.r('packageVersion("iglu")'))
print(f"iglu version: {iglu_version}")
print(f"iglu_py version: {version('iglu-py')}")
print(f"rpy2 version: {version('rpy2')}")

Python version: 3.11.10 (main, Oct  3 2024, 02:26:51) [Clang 14.0.6 ]
R version: [1] "R version 4.4.3 (2025-02-28)"

iglu version: [1] ‘4.2.2’

iglu_py version: 1.1.1
rpy2 version: 3.6.0


## Test on synthetic data

- Samples - every 5 min
- duration - 1h
- values [80,120] repeated for sampling duration

Expected hourly AUC = 100 mg.h/dL

In [5]:
hours = 1
dt0 = 5
samples = int(hours*60/dt0)
times = pd.date_range('2020-01-01', periods=samples, freq=f"{dt0}min")
glucose_values = [80,120]* int(samples/2)

syntheticdata = pd.DataFrame({
    'id': ['subject1'] * samples,
    'time': times,
    'gl': glucose_values
})

synthetic_iglu_auc_results  = iglu_py.auc(syntheticdata)
synthetic_iglu_auc_results

,id,hourly_auc
1,subject1,102.222222


**Note:** Incorrect AUC calculation is a result of CGMS2DayByDay function bugs:
- one sample shift in interpolation - results in 11 samples instead of 12
- actual_dates returns 2 dates instead of one

Additional suspicious code is in AUC itself: `day = rep(data_ip[[2]], 1440/dt0),` - IMHO it resample sequential gl to different days, instead of sequential sampling for each day before sampling for the next 


## Test on example data  

In [6]:
test_data = "../tests/data/example_data_5_subject.csv"
# load test data into DF
df = pd.read_csv(test_data, index_col=0)

iglu_auc_results  = iglu_py.auc(df)
iglu_auc_results = iglu_auc_results.round(0)

iglu_auc_results.rename(columns={'hourly_auc': 'IGLU AUC (mg*h/dL)'}, inplace=True)
iglu_auc_results = pd.merge(iglu_auc_results, ChatGPT_results[['id', 'ChatGPT AUC (mg*h/dL)']], on='id', how='left')
iglu_auc_results['Difference (%)'] = ((iglu_auc_results['IGLU AUC (mg*h/dL)'] - iglu_auc_results['ChatGPT AUC (mg*h/dL)']) / iglu_auc_results['ChatGPT AUC (mg*h/dL)'] * 100).round(1)


display(iglu_auc_results)


,id,IGLU AUC (mg*h/dL),ChatGPT AUC (mg*h/dL),Difference (%)
0,Subject 1,123.0,104.0,18.3
1,Subject 2,218.0,197.0,10.7
2,Subject 3,154.0,134.0,14.9
3,Subject 4,130.0,117.0,11.1
4,Subject 5,174.0,155.0,12.3


## Conclusions 
IGLU AUC calculations are substantially differ from expected ranges suggested by ChatGPT


# IGLU_PYTHON results

In [7]:
# Add project directory to PYTHONPATH
import os
import sys
import pandas as pd
sys.path.append(os.path.abspath('..'))
import iglu_python


## Test on synthetic data

In [8]:
synthetic_iglu_auc_results  = iglu_python.auc(syntheticdata)
synthetic_iglu_auc_results

,id,hourly_auc
0,subject1,100.0


**Note:** Result match expected

## Test on Example data

In [9]:
# load test data into DF
df = pd.read_csv(test_data, index_col=0)

iglu_python_auc_results = iglu_python.auc(df)
iglu_python_auc_results = iglu_python_auc_results.round(0)

iglu_python_auc_results.rename(columns={'hourly_auc': 'IGLU PYTHON AUC (mg*h/dL)'}, inplace=True)
iglu_python_auc_results = pd.merge(iglu_python_auc_results, iglu_auc_results[['id', 'IGLU AUC (mg*h/dL)']], on='id', how='left')
iglu_python_auc_results = pd.merge(iglu_python_auc_results, ChatGPT_results[['id', 'ChatGPT AUC (mg*h/dL)']], on='id', how='left')
iglu_python_auc_results['Difference to IGLU(%)'] = ((iglu_python_auc_results['IGLU PYTHON AUC (mg*h/dL)'] - iglu_python_auc_results['IGLU AUC (mg*h/dL)']) / iglu_python_auc_results['IGLU AUC (mg*h/dL)'] * 100).round(1)
iglu_python_auc_results['Difference to ChatGPt(%)'] = ((iglu_python_auc_results['IGLU PYTHON AUC (mg*h/dL)'] - iglu_python_auc_results['ChatGPT AUC (mg*h/dL)']) / iglu_python_auc_results['ChatGPT AUC (mg*h/dL)'] * 100).round(1)

display(iglu_python_auc_results)


,id,IGLU PYTHON AUC (mg*h/dL),IGLU AUC (mg*h/dL),ChatGPT AUC (mg*h/dL),Difference to IGLU(%),Difference to ChatGPt(%)
0,Subject 1,123.0,123.0,104.0,0.0,18.3
1,Subject 2,219.0,218.0,197.0,0.5,11.2
2,Subject 3,154.0,154.0,134.0,0.0,14.9
3,Subject 4,130.0,130.0,117.0,0.0,11.1
4,Subject 5,175.0,174.0,155.0,0.6,12.9


## Conclusions  
IGLU_PYTHON AUC calculations are close to IGLU calculations (-0.5%)

